In [ ]:
!pip install rawpy imageio exifread

import numpy as np
import matplotlib.pyplot as plt

from google.colab import files
uploaded = files.upload()

In [ ]:
import rawpy
import matplotlib.pyplot as plt
################################################################################
def load_image(file_name= 'MIT_FiveK_5.dng'):
    """
    """

    with rawpy.imread(file_name) as raw:

        bl = raw.black_level_per_channel
        print("Black Level per channel:", bl)
        pattern = raw.raw_pattern
        bayer_img = raw.raw_image_visible
        rgb_image= raw.postprocess()

        color_map = {0: 'R', 1: 'G', 2: 'B', 3: 'G'}
        named_pattern = np.vectorize(color_map.get)(pattern)
        print(named_pattern)

        # Convert embedded thumbnail to NumPy array
        thumbnail = raw.extract_thumb()
        print(thumbnail.format) # we this later
        if thumbnail.format == rawpy.ThumbFormat.JPEG:
            img = Image.open(io.BytesIO(thumbnail.data))
            thumbnail_np = np.array(img.convert("L"))  # convert to grayscale
        elif thumbnail.format == rawpy.ThumbFormat.BITMAP:
            thumbnail_np = np.array(thumbnail.data)
        else:
            raise ValueError("Unknown thumbnail format")

    return bayer_img.astype(np.uint16), bl, thumbnail_np

################################################################################
def BLC_RGGB(image,bl):
    bl_r, bl_gr, bl_gb, bl_b = bl

    H, W = image.shape
    print(H, W)

    corrected = image.astype(np.int32)
    corrected[0::2, 0::2] -= bl_r   # R
    corrected[0::2, 1::2] -= bl_gr  # Gr
    corrected[1::2, 0::2] -= bl_gb  # Gb
    corrected[1::2, 1::2] -= bl_b   # B

    BLC_image = np.clip(corrected, 0, None).astype(np.uint16)
    return BLC_image

In [ ]:
import cv2

bayer_img, bl, thumbnail_np = load_image(file_name= 'MIT_FiveK_5.dng')

BLC_image = BLC_RGGB(bayer_img,bl)

BLC_image_rotated = cv2.rotate(BLC_image, cv2.ROTATE_90_COUNTERCLOCKWISE)
bayer_img_rotated= cv2.rotate(bayer_img, cv2.ROTATE_90_COUNTERCLOCKWISE)

fig, axes = plt.subplots(1,2,figsize=(7, 5))
axes[0].imshow(bayer_img_rotated, cmap='gray')
axes[0].set_title('Bayer Image')
axes[0].axis('off')
axes[1].imshow(BLC_image_rotated, cmap='gray')
axes[1].set_title('Black Level Corrected Image')
axes[1].axis('off')

plt.tight_layout(pad=0)
plt.show()
